# Technical test for Worldstores.co.uk

This is my ipython notebook to display the results of my technical test for worldstores.co.uk. I did the analysis using pandas, mostly because Kuba mentioned that it would be a useful thing to have used and I wanted to demonstrate that I could pick it up fairly quickly.

The code is python2.7

If you accidentally double-click on a cell you will see the markdown source for the text, just press ctrl-enter to run it and the formatted text should re-appear.

## The assignment

1.	For every site and order_type, by month (ts_placed is when order was placed) show the following:
    *	number of orders
    *	number of customers
    *	Current revenue (use order_total_incVat)
    *	Original revenue (use original_order_value_incVAt)
    *	Average basket size (number of distinct items on an order)
    *	What is goods only revenue? (order_total_incVAT includes postage charge)
    *	Is there anything to note about postage we charged the customers throughout the year?
2.	In which month were refunds the highest? (assume refunds are the difference between original order value and current order total)
3.	Which site is the best and worst in terms of refunds?
4.	Use the product properties to see which classes (use Class1) of products have the highest sales and least refunds proportion.
5.	Which suppliers (supplier_id) are the best and worst? Motivate your answer.

For the next questions it would be useful to calculate two more columns and add them to fact_order but you can propose an alternative solution. Make sure that your solution is scalable.

*	Cumulative order count per customer
*	Recency, i.e. days elapsed from previous order of the same customer

Example: <img src="example.png" width="700" height="200"/>

6.	Show for every site what percentage of customers bought more than once.
7.	Show for every site the percentage of customers who made their second purchase within 30 days
8.	Are there any anomalies in the data? 

Preferred solution will be in SQL or Python.
Please provide the code. Please make sure that question 1 can be answered for any same-structured data set by running your script/scripts.



My code:

```python
"""Some code to prove that I can. Also, I'm going to learn Pandas"""

import pandas as pd
import numpy as np
import os

directory = "/home/simon/Documents/WStest/"


def print_headers(file):
    data = pd.read_csv(directory+file, low_memory=False)
    print file.split("/")[-1]
    print(list(data))


def array_uniques(array):
    return len(np.unique(array))

#for file in os.listdir(directory):
#    if ".csv" in file:
#        print_headers(file)

# Output the number of rows
# print("Total rows: {0}".format(len(data)))
# See which headers are available


# For every site and order_type, by month (ts_placed is when order was placed) show the following:

data = pd.read_csv(directory + "fact_order.csv", parse_dates=['ts_placed'])

data2 = pd.read_csv(directory + "fact_orderline.csv")

# print data.head(1)

data["month_placed"] = [m.month for m in data["ts_placed"]]

data2["goods_value"] = data2["current_quantity"] * \
                                data2["item_price_incVAT_per_unit"]

tojoin = data2.groupby('order_idno', as_index=False).agg(
    {'original_quantity': sum,
     'current_quantity': sum,
     'goods_value': sum})

result = pd.concat([data, tojoin], axis=1, join='inner')

result = result.loc[:, ~result.columns.duplicated()] # you learn new things everyday.

final = result[["order_idno",
                "order_type",
                "sitename",
                "customer_idno",
                "order_total_incVAT",
                "month_placed",
                "current_quantity",
                "goods_value",
                'original_order_value_incVAT'
                ]].groupby(['sitename',
                           'order_type',
                           'month_placed']).agg(
    {'order_idno':np.size,
     'customer_idno': array_uniques,
     'order_total_incVAT': sum,
     'original_order_value_incVAT':sum,
     'current_quantity': np.mean,
     'goods_value': sum
    })

print final


```

## Full results

the python command

```python
print formatted_data.to_html()

```
returns this table (markdown recognises html table formatting).

Note that there is a scroll bar at the bottom as this is a large and clunky table.

It displays all of the information required in part one of this exercise, but would realistically be sliced to give specific infomation.

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th></th>
      <th></th>
      <th>current_revenue</th>
      <th>original_revenue</th>
      <th>number_of_customers</th>
      <th>number_of_orders</th>
      <th>average_basket_size</th>
      <th>goods_value</th>
    </tr>
    <tr>
      <th>sitename</th>
      <th>order_type</th>
      <th>month_placed</th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th rowspan="10" valign="top">www.bedroomworld.co.uk</th>
      <th rowspan="10" valign="top">online</th>
      <th>1</th>
      <td>107,179.51</td>
      <td>117,510.96</td>
      <td>458</td>
      <td>463</td>
      <td>1.62</td>
      <td>101,153.85</td>
    </tr>
    <tr>
      <th>2</th>
      <td>104,452.10</td>
      <td>111,349.58</td>
      <td>408</td>
      <td>414</td>
      <td>1.69</td>
      <td>98,758.08</td>
    </tr>
    <tr>
      <th>3</th>
      <td>120,270.35</td>
      <td>126,251.38</td>
      <td>438</td>
      <td>441</td>
      <td>1.80</td>
      <td>114,045.50</td>
    </tr>
    <tr>
      <th>4</th>
      <td>99,877.84</td>
      <td>106,037.21</td>
      <td>379</td>
      <td>381</td>
      <td>1.75</td>
      <td>94,564.87</td>
    </tr>
    <tr>
      <th>5</th>
      <td>83,213.49</td>
      <td>90,779.81</td>
      <td>309</td>
      <td>309</td>
      <td>1.79</td>
      <td>79,688.52</td>
    </tr>
    <tr>
      <th>6</th>
      <td>68,509.84</td>
      <td>73,037.26</td>
      <td>293</td>
      <td>294</td>
      <td>1.65</td>
      <td>68,414.92</td>
    </tr>
    <tr>
      <th>7</th>
      <td>87,696.36</td>
      <td>98,491.36</td>
      <td>378</td>
      <td>379</td>
      <td>1.54</td>
      <td>87,631.42</td>
    </tr>
    <tr>
      <th>8</th>
      <td>91,626.91</td>
      <td>99,437.56</td>
      <td>386</td>
      <td>388</td>
      <td>1.65</td>
      <td>91,451.91</td>
    </tr>
    <tr>
      <th>9</th>
      <td>85,367.45</td>
      <td>96,030.49</td>
      <td>355</td>
      <td>355</td>
      <td>1.70</td>
      <td>85,142.46</td>
    </tr>
    <tr>
      <th>10</th>
      <td>57,857.82</td>
      <td>65,049.13</td>
      <td>250</td>
      <td>250</td>
      <td>1.59</td>
      <td>57,702.88</td>
    </tr>
    <tr>
      <th rowspan="20" valign="top">www.kiddicare.com</th>
      <th rowspan="10" valign="top">online</th>
      <th>1</th>
      <td>191,355.57</td>
      <td>197,342.43</td>
      <td>2500</td>
      <td>2522</td>
      <td>1.67</td>
      <td>185,935.27</td>
    </tr>
    <tr>
      <th>2</th>
      <td>127,139.02</td>
      <td>130,945.87</td>
      <td>1733</td>
      <td>1743</td>
      <td>1.64</td>
      <td>123,047.62</td>
    </tr>
    <tr>
      <th>3</th>
      <td>140,672.27</td>
      <td>146,197.01</td>
      <td>1846</td>
      <td>1860</td>
      <td>1.67</td>
      <td>136,258.67</td>
    </tr>
    <tr>
      <th>4</th>
      <td>151,331.23</td>
      <td>157,254.06</td>
      <td>2045</td>
      <td>2054</td>
      <td>1.74</td>
      <td>146,797.39</td>
    </tr>
    <tr>
      <th>5</th>
      <td>138,612.53</td>
      <td>149,396.78</td>
      <td>1658</td>
      <td>1662</td>
      <td>1.72</td>
      <td>134,868.25</td>
    </tr>
    <tr>
      <th>6</th>
      <td>115,101.68</td>
      <td>122,870.85</td>
      <td>1570</td>
      <td>1581</td>
      <td>1.64</td>
      <td>111,562.50</td>
    </tr>
    <tr>
      <th>7</th>
      <td>161,258.88</td>
      <td>170,174.90</td>
      <td>2151</td>
      <td>2165</td>
      <td>1.64</td>
      <td>156,366.08</td>
    </tr>
    <tr>
      <th>8</th>
      <td>136,215.32</td>
      <td>146,211.71</td>
      <td>1851</td>
      <td>1859</td>
      <td>1.63</td>
      <td>132,131.20</td>
    </tr>
    <tr>
      <th>9</th>
      <td>120,997.34</td>
      <td>127,267.46</td>
      <td>1677</td>
      <td>1685</td>
      <td>1.59</td>
      <td>117,137.62</td>
    </tr>
    <tr>
      <th>10</th>
      <td>98,641.13</td>
      <td>101,136.08</td>
      <td>1326</td>
      <td>1327</td>
      <td>1.64</td>
      <td>95,639.77</td>
    </tr>
    <tr>
      <th rowspan="10" valign="top">shop</th>
      <th>1</th>
      <td>24,818.38</td>
      <td>25,470.37</td>
      <td>97</td>
      <td>132</td>
      <td>1.27</td>
      <td>24,818.38</td>
    </tr>
    <tr>
      <th>2</th>
      <td>21,790.78</td>
      <td>21,790.78</td>
      <td>86</td>
      <td>122</td>
      <td>1.39</td>
      <td>21,790.78</td>
    </tr>
    <tr>
      <th>3</th>
      <td>24,170.30</td>
      <td>24,170.30</td>
      <td>73</td>
      <td>103</td>
      <td>1.34</td>
      <td>24,170.30</td>
    </tr>
    <tr>
      <th>4</th>
      <td>22,108.82</td>
      <td>22,857.82</td>
      <td>86</td>
      <td>119</td>
      <td>1.32</td>
      <td>22,108.82</td>
    </tr>
    <tr>
      <th>5</th>
      <td>19,323.96</td>
      <td>20,022.96</td>
      <td>74</td>
      <td>105</td>
      <td>1.33</td>
      <td>19,323.96</td>
    </tr>
    <tr>
      <th>6</th>
      <td>16,171.45</td>
      <td>16,561.45</td>
      <td>64</td>
      <td>86</td>
      <td>1.24</td>
      <td>16,171.45</td>
    </tr>
    <tr>
      <th>7</th>
      <td>19,170.98</td>
      <td>19,176.37</td>
      <td>80</td>
      <td>109</td>
      <td>1.23</td>
      <td>19,170.98</td>
    </tr>
    <tr>
      <th>8</th>
      <td>20,783.45</td>
      <td>20,783.45</td>
      <td>89</td>
      <td>112</td>
      <td>1.21</td>
      <td>20,783.45</td>
    </tr>
    <tr>
      <th>9</th>
      <td>14,139.70</td>
      <td>14,139.70</td>
      <td>72</td>
      <td>94</td>
      <td>1.38</td>
      <td>14,139.70</td>
    </tr>
    <tr>
      <th>10</th>
      <td>20,423.35</td>
      <td>20,423.35</td>
      <td>66</td>
      <td>86</td>
      <td>1.64</td>
      <td>20,423.35</td>
    </tr>
    <tr>
      <th rowspan="10" valign="top">www.worldstores.co.uk</th>
      <th rowspan="10" valign="top">online</th>
      <th>1</th>
      <td>150,168.58</td>
      <td>160,927.17</td>
      <td>781</td>
      <td>785</td>
      <td>1.64</td>
      <td>141,807.85</td>
    </tr>
    <tr>
      <th>2</th>
      <td>149,923.69</td>
      <td>172,880.68</td>
      <td>725</td>
      <td>738</td>
      <td>1.73</td>
      <td>141,555.33</td>
    </tr>
    <tr>
      <th>3</th>
      <td>160,924.40</td>
      <td>176,754.79</td>
      <td>787</td>
      <td>793</td>
      <td>1.67</td>
      <td>151,991.47</td>
    </tr>
    <tr>
      <th>4</th>
      <td>142,733.78</td>
      <td>164,632.24</td>
      <td>763</td>
      <td>766</td>
      <td>1.62</td>
      <td>134,506.86</td>
    </tr>
    <tr>
      <th>5</th>
      <td>101,536.95</td>
      <td>120,008.72</td>
      <td>574</td>
      <td>581</td>
      <td>1.46</td>
      <td>96,084.61</td>
    </tr>
    <tr>
      <th>6</th>
      <td>99,467.27</td>
      <td>121,105.99</td>
      <td>541</td>
      <td>550</td>
      <td>1.65</td>
      <td>99,043.08</td>
    </tr>
    <tr>
      <th>7</th>
      <td>119,824.38</td>
      <td>139,487.96</td>
      <td>630</td>
      <td>637</td>
      <td>1.64</td>
      <td>119,405.11</td>
    </tr>
    <tr>
      <th>8</th>
      <td>144,829.45</td>
      <td>171,615.83</td>
      <td>749</td>
      <td>750</td>
      <td>1.58</td>
      <td>144,280.31</td>
    </tr>
    <tr>
      <th>9</th>
      <td>110,142.04</td>
      <td>131,319.11</td>
      <td>626</td>
      <td>631</td>
      <td>1.57</td>
      <td>109,617.80</td>
    </tr>
    <tr>
      <th>10</th>
      <td>111,674.99</td>
      <td>126,281.51</td>
      <td>565</td>
      <td>569</td>
      <td>1.66</td>
      <td>111,235.74</td>
    </tr>
  </tbody>
</table>

## Postage

To calculate the postage from the previous table, I used the following python code:
```python
postage_table = (formatted_data["current_revenue"] - formatted_data["goods_value"]).to_frame("postage")
```
exporting as html once again yields the following table:

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th></th>
      <th></th>
      <th>postage</th>
    </tr>
    <tr>
      <th>sitename</th>
      <th>order_type</th>
      <th>month_placed</th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th rowspan="10" valign="top">www.bedroomworld.co.uk</th>
      <th rowspan="10" valign="top">online</th>
      <th>1</th>
      <td>6025.66</td>
    </tr>
    <tr>
      <th>2</th>
      <td>5694.02</td>
    </tr>
    <tr>
      <th>3</th>
      <td>6224.85</td>
    </tr>
    <tr>
      <th>4</th>
      <td>5312.97</td>
    </tr>
    <tr>
      <th>5</th>
      <td>3524.97</td>
    </tr>
    <tr>
      <th>6</th>
      <td>94.92</td>
    </tr>
    <tr>
      <th>7</th>
      <td>64.94</td>
    </tr>
    <tr>
      <th>8</th>
      <td>175.00</td>
    </tr>
    <tr>
      <th>9</th>
      <td>224.99</td>
    </tr>
    <tr>
      <th>10</th>
      <td>154.94</td>
    </tr>
    <tr>
      <th rowspan="20" valign="top">www.kiddicare.com</th>
      <th rowspan="10" valign="top">online</th>
      <th>1</th>
      <td>5420.30</td>
    </tr>
    <tr>
      <th>2</th>
      <td>4091.40</td>
    </tr>
    <tr>
      <th>3</th>
      <td>4413.60</td>
    </tr>
    <tr>
      <th>4</th>
      <td>4533.84</td>
    </tr>
    <tr>
      <th>5</th>
      <td>3744.28</td>
    </tr>
    <tr>
      <th>6</th>
      <td>3539.18</td>
    </tr>
    <tr>
      <th>7</th>
      <td>4892.80</td>
    </tr>
    <tr>
      <th>8</th>
      <td>4084.12</td>
    </tr>
    <tr>
      <th>9</th>
      <td>3859.72</td>
    </tr>
    <tr>
      <th>10</th>
      <td>3001.36</td>
    </tr>
    <tr>
      <th rowspan="10" valign="top">shop</th>
      <th>1</th>
      <td>0.00</td>
    </tr>
    <tr>
      <th>2</th>
      <td>0.00</td>
    </tr>
    <tr>
      <th>3</th>
      <td>0.00</td>
    </tr>
    <tr>
      <th>4</th>
      <td>0.00</td>
    </tr>
    <tr>
      <th>5</th>
      <td>0.00</td>
    </tr>
    <tr>
      <th>6</th>
      <td>0.00</td>
    </tr>
    <tr>
      <th>7</th>
      <td>0.00</td>
    </tr>
    <tr>
      <th>8</th>
      <td>0.00</td>
    </tr>
    <tr>
      <th>9</th>
      <td>0.00</td>
    </tr>
    <tr>
      <th>10</th>
      <td>0.00</td>
    </tr>
    <tr>
      <th rowspan="10" valign="top">www.worldstores.co.uk</th>
      <th rowspan="10" valign="top">online</th>
      <th>1</th>
      <td>8360.73</td>
    </tr>
    <tr>
      <th>2</th>
      <td>8368.36</td>
    </tr>
    <tr>
      <th>3</th>
      <td>8932.93</td>
    </tr>
    <tr>
      <th>4</th>
      <td>8226.92</td>
    </tr>
    <tr>
      <th>5</th>
      <td>5452.34</td>
    </tr>
    <tr>
      <th>6</th>
      <td>424.19</td>
    </tr>
    <tr>
      <th>7</th>
      <td>419.27</td>
    </tr>
    <tr>
      <th>8</th>
      <td>549.14</td>
    </tr>
    <tr>
      <th>9</th>
      <td>524.24</td>
    </tr>
    <tr>
      <th>10</th>
      <td>439.25</td>
    </tr>
  </tbody>
</table>

or as a graph using python's matplotlib library:

<img src="postage.png"/>

which shows that postage was reduced significantly in June for the worldstores and bedroomworld websites.

Further exploration of the data (along with cheating but looking at the postage deals on the website!) shows that 96.3% of orders from the worldstores site which
1. were placed after June and
2. paid no postage
had a goods value greater than £50 - suggesting that this is when that offer was implemented.

## Refunds

First we group refunds by month, regardless of site. The question asks in which month refunds were the highest, I considered both the absolute value, and that value as a percentage of revenue:

Although January and March have higher absolute values of refunds, February has a higher proportion of it's revenue lost in refunds.

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>order_total_incVAT</th>
      <th>refunds</th>
      <th>refund_value_as_percent</th>
    </tr>
    <tr>
      <th>month_placed</th>
      <th></th>
      <th></th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>1</th>
      <td>473522.04</td>
      <td>19806.69</td>
      <td>4.182844</td>
    </tr>
    <tr>
      <th>2</th>
      <td>403305.59</td>
      <td>18153.78</td>
      <td>4.501247</td>
    </tr>
    <tr>
      <th>3</th>
      <td>446037.32</td>
      <td>19571.38</td>
      <td>4.387835</td>
    </tr>
    <tr>
      <th>4</th>
      <td>416051.67</td>
      <td>18073.73</td>
      <td>4.344107</td>
    </tr>
    <tr>
      <th>5</th>
      <td>342686.93</td>
      <td>12721.59</td>
      <td>3.712307</td>
    </tr>
    <tr>
      <th>6</th>
      <td>299250.24</td>
      <td>4058.29</td>
      <td>1.356153</td>
    </tr>
    <tr>
      <th>7</th>
      <td>387950.60</td>
      <td>5377.01</td>
      <td>1.386004</td>
    </tr>
    <tr>
      <th>8</th>
      <td>393455.13</td>
      <td>4808.26</td>
      <td>1.222061</td>
    </tr>
    <tr>
      <th>9</th>
      <td>330646.53</td>
      <td>4608.95</td>
      <td>1.393921</td>
    </tr>
    <tr>
      <th>10</th>
      <td>288597.29</td>
      <td>3595.55</td>
      <td>1.245871</td>
    </tr>
  </tbody>
</table>

Similarly we can look at this data by site name:

Worldstores refunds the most value by a hair, but when considered as a fraction of the revenue it definitely refunds more than the other brands.

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>order_total_incVAT</th>
      <th>refunds</th>
      <th>refund_value_as_percent</th>
    </tr>
    <tr>
      <th>sitename</th>
      <th></th>
      <th></th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>www.bedroomworld.co.uk</th>
      <td>906051.67</td>
      <td>27497.26</td>
      <td>3.034845</td>
    </tr>
    <tr>
      <th>www.kiddicare.com</th>
      <td>1584226.14</td>
      <td>41580.60</td>
      <td>2.624663</td>
    </tr>
    <tr>
      <th>www.worldstores.co.uk</th>
      <td>1291225.53</td>
      <td>41697.37</td>
      <td>3.229286</td>
    </tr>
  </tbody>
</table>

Grouping this data further shows that actually if we consider online seperately from in shop we see that all sites perform approximately equally kiddicare only appeared to outperform the other sites because its revenue was counted but not its refunds.

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th></th>
      <th>order_total_incVAT</th>
      <th>refunds</th>
      <th>refund_value_as_percent</th>
    </tr>
    <tr>
      <th>sitename</th>
      <th>order_type</th>
      <th></th>
      <th></th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>www.bedroomworld.co.uk</th>
      <th>online</th>
      <td>906,051.67</td>
      <td>27,497.26</td>
      <td>3.03</td>
    </tr>
    <tr>
      <th rowspan="2" valign="top">www.kiddicare.com</th>
      <th>online</th>
      <td>1,381,324.97</td>
      <td>41,580.60</td>
      <td>3.01</td>
    </tr>
    <tr>
      <th>shop</th>
      <td>202,901.17</td>
      <td>-0.00</td>
      <td>-0.00</td>
    </tr>
    <tr>
      <th>www.worldstores.co.uk</th>
      <th>online</th>
      <td>1,291,225.53</td>
      <td>41,697.37</td>
      <td>3.23</td>
    </tr>
  </tbody>
</table>